# 代码仓库：https://github.com/syb-5213/AssocRules

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import  linear_model
import itertools
import math

# 1.对数据集进行处理，转换成适合进行关联规则挖掘的形式

读取数据并打印所有属性

In [4]:
data = pd.read_csv('cbg_patterns.csv')

属性有: census_block_group、date_range_start、date_range_end、raw_visit_count、raw_visitor_count、visitor_home_cbgs、visitor_work_cbgs、distance_from_home、related_same_day_brand、related_same_month_brand、top_brands、popularity_by_hour、popularity_by_day

选择属性related_same_day_brand（属性related_same_month_brand以及top_brands处理过程类似）进行关联规则的挖掘,首先将数据集变为易于挖掘关联规则的格式,代码如下

In [5]:
a=data["related_same_day_brand"]
all_data=[]
for i in range(len(a)):
    b=a.iloc[[i]].values[0]
    c=b[1:len(b)-1]
    if(not(len(c)==0)):
        t_data=[]
        d=c.split(',')
        for j in range(len(d)):
            e=d[j][1:len(d[j])-1]
            t_data.append(e)
        all_data.append(t_data)

转换格式后的数据保存在all_data变量中，如下列出一些转换后的数据格式，每一项记录代表一个人访问过哪些品牌

In [7]:
for i in range(len(all_data)):
    if(i<10):
        print(all_data[i])
    else:
        break

['Chick-fil-A', 'mcdonalds', 'Marathon Petroleum', 'walmart']
['Shell Oil', 'mcdonalds', 'Chick-fil-A', 'Chevron']
['Dollar General']
['Chick-fil-A', "Sam's Club", 'Dollar General', 'walmart']
['Chevron', 'Daylight Donuts', 'walmart']
['walmart']
['walmart', 'Chick-fil-A']
['The American Legion', 'Dollar General', "Jack's Family Restaurants"]
["Papa Murphy's", 'starbucks', 'Holiday Station']
['Burger King US', 'ConocoPhillips', 'SUBWAY', 'Chevron', 'Ace Hardware', 'Shell Oil', 'mcdonalds', "Denny's"]


# 2.找出频繁模式

由于数据项比较多，采用fp树的方式进行频繁模式的挖掘


首先对上述处理后的数据进一步压缩，合并相同项集并用count记录该项集出现的次数，代码如create_comb_data函数所示。

In [10]:
def create_comb_data(data):
    comb_data={}
    for i in data:
        key = frozenset(i)
        if key in comb_data:
            comb_data[frozenset(i)] += 1
        else:
            comb_data[frozenset(i)] = 1
    return comb_data
comb_data = create_comb_data(all_data)

创建fp树的结点类node，其中name用于存放结点名字，count用于计数，nodeLink用于连接相似结点，parent用于存放父节点，用于回溯，children存放儿子结点。

In [11]:
class node:
    def __init__(self, name, counts, parent_node):
        self.name = name
        self.count = counts
        self.nodeLink = None
        self.parent = parent_node
        self.children = {}

    def add(self, counts):
        self.count += counts

接下来开始构建fp树如函数crerat_fp_tree所示，参数minSupport表示最小支持度（为方便起见这里以及后面采用次数作为支持度而不用比率），代码中的注释详细解释了每部分的作用，其中update_FPtree函数递归的生成课件中的纵向链表，update_item_list函数递归的生成课件中每个table表中item的横向链表。

In [12]:
def update_item_list(node1, node2):
    while node1.nodeLink != None:
        node1 = node1.nodeLink
    node1.nodeLink = node2
def update_FPtree(items, tree, item_list, count):
    if items[0] in tree.children:
        #判断items的第一个结点是否已作为子结点
        tree.children[items[0]].add(count)
    else:
        #创建新的分支
        tree.children[items[0]] = node(items[0], count, tree)
        #更新相应频繁项集的链表，往后添加
        if item_list[items[0]][1] == None:
            item_list[items[0]][1] = tree.children[items[0]]
        else:
            update_item_list(item_list[items[0]][1], tree.children[items[0]])
    #递归
    if len(items) > 1:
        update_FPtree(items[1::], tree.children[items[0]], item_list, count)

def crerat_fp_tree(data, minSupport):
    #创建每个频繁item的列表item_list
    item_list = {}
    #根据数据集data汇集每个item数量
    for i in data:
        for j in i:
            item_list[j] = item_list.get(j, 0)+data[i]
    #将数据项小于min_support删除掉
    for k in list(item_list.keys()):
        if item_list[k] < minSupport:
            del(item_list[k])
    #创建频繁项集的索引
    frequent_item = set(item_list.keys())
    #将item_list的格式改为值及指向下一个item出现处的指针
    for k in item_list:
        item_list[k] = [item_list[k], None]
    #创建FP树的树根tree结点
    tree = node('', 1, None)
    for i,count in data.items():
        #首先筛选出满足最小支持度的项集
        item = {}
        for j in i:
            if j in frequent_item: 
                item[j] = item_list[j][0] # element : count
        if len(item) > 0:
            #按照频数对每个item项集进行排序并更新fp树
            ordered_item = [v[0] for v in sorted(item.items(), key=lambda p:(p[1], -ord(p[0][0])), reverse=True)]
            update_FPtree(ordered_item, tree, item_list, count)
    return tree, item_list

根据每个项集出现的频数，设置最小支持度minSupport为100进行fp树的构建，树根结点为变量tree，频繁项集表头指针存放在变量item_list中。

In [13]:
minSupport=100
tree, item_list = crerat_fp_tree(comb_data, minSupport)

按照课件上给定步骤构造指定item的条件模式基，代码如函数create_conditional_database所示。

In [14]:
def create_conditional_database(item, item_list):
    #tree_node变量指向传参item在FP树中的第一个结点
    tree_node = item_list[item][1] 
    conditional_database={}
    #递归的沿着tree_node结点向上寻找一条到根节点的路径并保存在path中，之后在进行下一个item位置继续寻找path
    while tree_node != None:
        path = []
        temp_node=tree_node
        if(temp_node.parent!=None):
            while temp_node.parent!=None:
                path.append(temp_node.name)
                temp_node=temp_node.parent
            if len(path) > 1:
                conditional_database[frozenset(path[1:])] = tree_node.count
            tree_node = tree_node.nodeLink 
    return conditional_database

对每一个频繁项都建立一棵条件FP树，然后对每个条件FP树递归地挖掘，最后可以得到所有满足最小支持度的频繁项集，代码如函数mine_FPtree所示。

In [15]:
def mine_FPtree(tree, item_list, minSupport, l, result):
    #最开始的频繁项集是item_list中的各元素
    t = [v[0] for v in sorted(item_list.items(), key=lambda p:p[1][0])]
    #对每个频繁项进行处理
    for i in t: 
        new_t = l.copy()
        new_t.append(i)
        if(len(new_t)>1):
            result.append([new_t,item_list[i][0]])
        #当前频繁项集的条件模式基
        conditional_database_i = create_conditional_database(i, item_list) 
        #构造当前频繁项的条件FP树
        new_tree, new_item_list = crerat_fp_tree(conditional_database_i, minSupport) 
        if new_item_list != None:
            #递归挖掘条件FP树
            mine_FPtree(new_tree, new_item_list, minSupport, new_t, result) 

按照最小支持度minSupport=100的要求提取的所有频繁项集保存在freItems列表中，总共有2835个满足要求的频繁项集。

In [16]:
freqItems = []
mine_FPtree(tree, item_list, minSupport, list([]), freqItems)

In [ ]:
筛选出其中一部分频繁项集以及其支持度进行展示，如下所示。

In [17]:
for i in range(len(freqItems)):
    if(i<20):
        print(freqItems[i][0],freqItems[i][1])
    else:
        break

['Freightliner Trucks', 'Pilot Travel Centers'] 101
['Freightliner Trucks', 'TravelCenters of America'] 108
["Buddy's", 'Stripes Convenience Stores'] 119
['Harps Food Store', 'Sonic'] 112
['Harps Food Store', 'Sonic', 'walmart'] 105
['Harps Food Store', 'walmart'] 119
['Acme Markets', 'Wawa'] 105
['Bomgaars', "Casey's General Stores"] 120
['County Market', "Casey's General Stores"] 108
['Simple Simon‚Äôs Pizza', 'Sonic'] 109
['Simple Simon‚Äôs Pizza', 'walmart'] 110
['Pet Pros', 'starbucks'] 111
["Spencer's", 'Wegmans Food Markets'] 125
["Coborn's", 'Holiday Station'] 103
['OnCue', 'walmart'] 120
['OnCue', 'Phillips 66'] 127
['Chrysler', 'Jeep'] 172
['Chrysler', 'Jeep', 'Dodge'] 170
['Chrysler', 'Dodge'] 178
['The Exchange', 'QuikTrip'] 146


# 3.导出关联规则，计算其支持度和置信度;

设置参数最小支持度为100、最小置信度为0.75提取关联规则，最终得到61组关联规则,其中每一条规则有4项，第一个为条件，第二项为全集，第三项为支持度，第四项为置信度，例如["Buddy's",["Buddy's",'Stripes Convenience Stores'], 119, 0.9296875]表示Buddy's->Stripes Convenience Stores[119,0.9296875]
。提取代码如下所示。

In [19]:
minSupport=100
minConfidence=0.75
assocrules = []
for i in range(len(freqItems)):
    t=freqItems[i][0]
    count=freqItems[i][1]
    for j in range(len(t)):
        item=t[j]
        count_item=item_list[item][0]
        if(count/count_item>minConfidence):
            assocrules.append([item,t,count,count/count_item])

筛选出其中一部分关联规则以及其支持度与置信度结果进行展示，如下所示。

In [20]:
for i in range(len(assocrules)):
    if(i<20):
        print(assocrules[i])
    else:
        break

['Freightliner Trucks', ['Freightliner Trucks', 'Pilot Travel Centers'], 101, 0.9017857142857143]
['Freightliner Trucks', ['Freightliner Trucks', 'TravelCenters of America'], 108, 0.9642857142857143]
["Buddy's", ["Buddy's", 'Stripes Convenience Stores'], 119, 0.9296875]
['Harps Food Store', ['Harps Food Store', 'Sonic'], 112, 0.8682170542635659]
['Harps Food Store', ['Harps Food Store', 'Sonic', 'walmart'], 105, 0.813953488372093]
['Harps Food Store', ['Harps Food Store', 'walmart'], 119, 0.9224806201550387]
['Acme Markets', ['Acme Markets', 'Wawa'], 105, 0.7894736842105263]
['Bomgaars', ['Bomgaars', "Casey's General Stores"], 120, 0.8759124087591241]
['County Market', ['County Market', "Casey's General Stores"], 108, 0.7659574468085106]
['Simple Simon‚Äôs Pizza', ['Simple Simon‚Äôs Pizza', 'Sonic'], 109, 0.7730496453900709]
['Simple Simon‚Äôs Pizza', ['Simple Simon‚Äôs Pizza', 'walmart'], 110, 0.7801418439716312]
['Pet Pros', ['Pet Pros', 'starbucks'], 111, 0.7655172413793103]
["Spenc

# 4.对规则进行评价

采用Lift以及Allconf两种指标对提取出来的关联规则进行评价，代码如下所示。

In [22]:
sum_number=len(all_data)
for i in range(len(assocrules)):
    sab=assocrules[i][2]/sum_number
    sa=item_list[assocrules[i][1][0]][0]/sum_number
    sb=item_list[assocrules[i][1][1]][0]/sum_number
    lift=sab/(sa*sb)
    allconf=(sab/sa+sab/sb)/2
    assocrules[i]=[assocrules[i][0],assocrules[i][1],assocrules[i][2],assocrules[i][3],lift,allconf]

筛选出其中一部分结果进行展示，其中lift以及Allconf评价指标结果见每一项后两个数所示。

In [23]:
for i in range(len(assocrules)):
    if(i<20):
        print(assocrules[i])
    else:
        break

['Freightliner Trucks', ['Freightliner Trucks', 'Pilot Travel Centers'], 101, 0.9017857142857143, 19.594556117061394, 0.4568180008715912]
['Freightliner Trucks', ['Freightliner Trucks', 'TravelCenters of America'], 108, 0.9642857142857143, 220.74037612572837, 0.5488919300723998]
["Buddy's", ["Buddy's", 'Stripes Convenience Stores'], 119, 0.9296875, 99.57872596153845, 0.499256705465587]
['Harps Food Store', ['Harps Food Store', 'Sonic'], 112, 0.8682170542635659, 15.768139740142452, 0.439600338448837]
['Harps Food Store', ['Harps Food Store', 'Sonic', 'walmart'], 105, 0.813953488372093, 14.78263100638355, 0.41212531729578467]
['Harps Food Store', ['Harps Food Store', 'walmart'], 119, 0.9224806201550387, 3.941602913764295, 0.46261311244012376]
['Acme Markets', ['Acme Markets', 'Wawa'], 105, 0.7894736842105263, 15.956018771144821, 0.40046640665835714]
['Bomgaars', ['Bomgaars', "Casey's General Stores"], 120, 0.8759124087591241, 17.62608352877632, 0.44447581754918064]
['County Market', ['Co

# 5.对挖掘结果进行分析

从提取出的关联规则中筛选其中两条规则进行分析。

In [25]:
temp=sorted(assocrules, key=lambda p:p[2], reverse=True)
print(temp[11]) 

['Cracker Barrel', ['Cracker Barrel', 'mcdonalds'], 622, 0.7794486215538847, 3.107140550862703, 0.39641867330573244]


（1）根据该条关联规则显示，支持度为622，置信度为77.9%，即既访问Cracker Barrel同时又访问mcdonalds的人数共有622人次，在访问Cracker Barrel的人群中有77.9%的人同样会访问mcdonalds，
该条规则的评价lift指标为3大于1表示具有正相关关系，而allconf指标显示为0.01小于0.5应该为负相关，这说明该规则受到零事务的影响导致lift评价指标出现问题，由该项规则可以发现lift指标受零事务的影响很大，而实际上我们去Cracker Barrel和去McDonald的相关性不应该取决于两者都不访问的记录，这一点正如课上所讲的lift和卡方指标的劣势，即容易受到数据记录大小的影响。

In [27]:
print(temp[36]) 

['Jeep', ['Jeep', 'Dodge'], 180, 0.9424083769633508, 863.9972007671972, 0.916748742937121]


（2）根据该条关联规则显示，支持度为180，置信度为94.2%，既访问Jeep同时又访问Dodge的人数共有188人次，在访问Jeep的人群中有94.2%的人同样会访问Dodge，该条规则的评价lift指标为863远远大于1表示具有正相关关系，而allconf指标显示为0.89大于0.5同样表示正相关关系，这说明jeep和dodge确实存在着强烈的正相关关系，而在实际中Jeep和Dodge是两款车的品牌都是克莱斯勒的产品，基本是同一个平台，所以两个品牌之间有非常强的共性，想购买Jeep车的人往往也会去Dodge看看，这一点比较贴合实际生活。所以可以在浏览Jeep的人群中宣传Dodge可能是一件比较有效率销售策略。